In [1]:
import pandas as pd
from xgboost import XGBClassifier
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('/content/drive/MyDrive/NLP_Course/imdb_train.csv')
dev = pd.read_csv('/content/drive/MyDrive/NLP_Course/imdb_val.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP_Course/imdb_test.csv')

In [3]:
def text_cleaning(text):
  text = re.sub("[^a-zA-Z]", " ", text) 
  text = text.lower()
  return text
train['review'] = train['review'].apply(text_cleaning)
dev['review'] = dev['review'].apply(text_cleaning)
test['review'] = test['review'].apply(text_cleaning)
corpus = train.review.tolist() + dev.review.tolist() + test.review.tolist()

In [4]:
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=10000)
tfidf.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [5]:
X_train, X_dev, X_test = tfidf.transform(train.review.tolist()).toarray(), tfidf.transform(dev.review.tolist()).toarray(), tfidf.transform(test.review.tolist()).toarray()
y_train, y_dev, y_test = train.sentiment.values, dev.sentiment.values, test.sentiment.values

In [6]:
BOW_XGB = XGBClassifier(max_depth=7, n_estimators=300, objective="binary:logistic", random_state=1, tree_method='gpu_hist', predictor='gpu_predictor')
BOW_XGB.fit(X_train, y_train, early_stopping_rounds=30, eval_set=[(X_dev, y_dev)])
y_pred = BOW_XGB.predict(X_test)

In [11]:
print(f"The test accuracy score is equal to {accuracy_score(y_test, y_pred)}")

The test accuracy score is equal to 0.865
